# Web Scraping

In [2]:
from bs4 import BeautifulSoup
import requests
from urllib.parse import urljoin 

def first_search_result(url):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    first_result = soup.find('td', 'result_text').find('a').get('href')

    return first_result
def movie_description(next_page_url2):
    headers = {'User-Agent': 'Mozilla/5.0'}
    response = requests.get(next_page_url2, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")
    #return soup.find("div", class_="summary_text").contents[0].strip()  # old version
    # class to be searhed may need to be changed depending on website updates. Just right click and inspect the description 
    # to get correct class.
    return soup.find("span", class_="sc-16ede01-1 kgphFu").get_text().strip()
def scrapeDescription(movieName):

    movieName = movieName.replace(' ','+')
    url = "http://www.imdb.com/find?ref_=nv_sr_fn&q=" + movieName + '&s=all'
    try:
        next_page = first_search_result(url)
    except:
        print(movieName)
        return " "
    next_page_url = urljoin(url, next_page)
    print("Webpage to be scraped: "+ next_page_url)
    try:
        return movie_description(next_page_url)
    except:
        print(movieName)
        return " "

movieName=input("Enter Movie Name\n> ")
print(scrapeDescription(movieName))

Enter Movie Name
> return of jedi
Webpage to be scraped: http://www.imdb.com/title/tt0086190/?ref_=fn_al_tt_1
After a daring mission to rescue Han Solo from Jabba the Hutt, the Rebels dispatch to Endor to destroy the second Death Star. Meanwhile, Luke struggles to help Darth Vader back from the dark side without falling into the Emperor's trap.


# Uploading Datasets

In [ ]:
# from google.colab import drive

# drive.mount('/content/gdrive')


In [3]:
import numpy as np
import pandas as pd
import pickle
import numpy as np
import tensorflow as tf
import random 
import keras
import time
total=time.time()
from keras.layers import Embedding
from keras.models import Model
from keras.layers import *
#!pip install keras_self_attention
from keras_self_attention import SeqSelfAttention
from keras.callbacks import ModelCheckpoint, EarlyStopping,LearningRateScheduler
from keras.initializers import TruncatedNormal
#create_embs
movies=pd.read_csv('data/movies_scraped.csv')
movies.description=movies.description+" "+movies.title+" "+movies.genre
#movies['embeddings']=movies.description.apply(lambda x: tokenizer.encode(str(x)))
movies['embeddings']=pickle.load( open( "data/embeds.pkl", "rb" ) )
ratings=pd.read_csv('data/ratings.csv')
##############user###########################
len(ratings.movie_id)
pd.set_option('mode.chained_assignment', None)

In [7]:
# userdata=pd.read_csv("ml-1m/users.dat",sep='::',header=None)
# userdata.head()
# userdata.columns=["user_id","gender","age_group","occupation","zipcode"]
# userdata.to_csv('users.csv',index=False)
ratings.head()

,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


In [8]:
movies.head()

,Unnamed: 0,movie_id,title,genre,description,embeddings
0,0,1,Toy Story (1995),Animation Children's Comedy,A cowboy doll is profoundly threatened and jea...,"[-0.20858312, 0.88775766, 0.3227467, -0.615751..."
1,1,2,Jumanji (1995),Adventure Children's Fantasy,When two kids find and play a magical board ga...,"[-0.17402853, -0.40350446, -0.31173387, 0.1971..."
2,2,3,Grumpier Old Men (1995),Comedy Romance,John and Max resolve to save their beloved bai...,"[0.65241635, 0.37577626, 0.10167889, 0.9934018..."
3,3,4,Waiting to Exhale (1995),Comedy Drama,"Based on Terry McMillan's novel, this film fol...","[0.27629364, 0.16062407, 0.09226728, -0.066186..."
4,4,5,Father of the Bride Part II (1995),Comedy,George Banks must deal not only with the pregn...,"[0.78778785, -0.5128158, -0.25456566, 0.018650..."


In [5]:
users=pd.read_csv("data/users.csv")
users.head()


,user_id,gender,age_group,occupation,zipcode
0,1,F,1,10,48067
1,2,M,56,16,70072
2,3,M,25,15,55117
3,4,M,45,7,02460
4,5,M,25,20,55455


# Item-Item Collaborative Filtering

In [9]:

def cos_sim(a,b):
    return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))

def pcc(a,b):
    return sum((a-a.mean())*(b-b.mean()))/(np.sqrt(sum((a-a.mean())**2))*np.sqrt(sum((b-b.mean())**2)))
def recommend(user_id,num_movies):
    i=0
    movie_list=ratings.loc[(ratings['user_id']==user_id  )]
    #movie_list['embeddings']=movie_list.apply(lambda x:tokenizer.encode(x.description))
    user_ratings=ratings.loc[ratings.user_id==user_id]
    preds=pd.DataFrame()
    for x in movies.itertuples():
        i+=1
        #print(i)
        if(x.movie_id in movie_list):
            continue
        else:
            emb=x.embeddings
            num=den=0
            for y in user_ratings.itertuples():
                #print(y.movie_id)
                #print(movies.loc[movies.movie_id==y.movie_id].embeddings.item())
                #sim=cos_sim(emb,movies.loc[movies.movie_id==y.movie_id].embeddings.item())
                sim=pcc(emb,movies.loc[movies.movie_id==y.movie_id].embeddings.item())
                num+=y.rating*sim
                den+=sim
            #print({'movie_id': x.movie_id,'title':x.title,'rating':num/den})
            temp=pd.DataFrame({'movie_id': [x.movie_id],'title':[x.title],'rating':[num/den]})
            #print(temp)
            preds=preds.append(temp, ignore_index=True)
            #print(preds.iloc[0])
    #scaling
    new_M=user_ratings.rating.max()
    new_m=user_ratings.rating.min()
    m=preds.rating.min()
    M=preds.rating.max()        
    preds.rating=preds.rating.apply(lambda x:(x-m)/(M-m)*(new_M-new_m)+new_m)        
    return preds.sort_values('rating',ascending=False).iloc[:num_movies]  

In [10]:
user_id = int(input("Enter user ID: "))
num_movies=int(input("Enter number of movies to be recommended "))
preds=recommend(user_id,num_movies)
preds.head()

Enter user ID: 6
Enter number of movies to be recommended 5


C:\Users\ASUS\AppData\Local\Temp\ipykernel_13556\3623836761.py:30: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  preds=preds.append(temp, ignore_index=True)


,movie_id,title,rating
3535,3604,Gypsy (1962),5.000000
1737,1797,Everest (1998),4.977381
2853,2922,Hang 'em High (1967),4.952669
2714,2783,"Tomb of Ligeia, The (1965)",4.933361
3086,3155,Anna and the King (1999),4.925806


# User-User Collaborative Filtering

In [11]:
#Dictionary of (userID, movieID): (rating)

ratingDict = {}
for row in ratings.itertuples():
    ratingDict[(row.user_id, row.movie_id)] = row.rating

In [12]:
import math
import numpy as np
import time

#PEARSON COEFFICIENT to calculate similarity between user_a and user_b
def returnSim(user_a, user_b):

    #find avg ratings given by user1 and user2
    avg_a = ratings.loc[(ratings['user_id']==user_a), 'rating'].mean()
    avg_b = ratings.loc[(ratings['user_id']==user_b), 'rating'].mean()

    sumNum = 0.0 #numerator of the final value in formula
    sumDen1 = 0.0 #1st denominator term
    sumDen2 = 0.0 #2nd denominator term

    for row in movies.itertuples():
        findMovie = row.movie_id

        rating_a = -1
        try:
            rating_a = ratingDict[(user_a, findMovie)]
        except:
            rating_a = -1

        rating_b = -1
        try:
            rating_b = ratingDict[user_b, findMovie]
        except:
            rating_b = -1

        if(not(rating_a == -1 or math.isnan(rating_a))): sumDen1 += pow((rating_a - avg_a),2)
        if(not(rating_b == -1 or math.isnan(rating_b))): sumDen2 += pow((rating_b - avg_b),2)
        if(not(rating_a == -1 or rating_b == -1 or math.isnan(rating_a) or math.isnan(rating_b))):
            sumNum += (rating_a - avg_a)*(rating_b - avg_b)

  
    if(sumDen1 == 0.0 or sumDen2 == 0.0):  
        return 0
    simValue = sumNum/( math.sqrt(sumDen1) * math.sqrt(sumDen2)) 
    return simValue


In [13]:
#Returns similarity of given user to every other user in form of dict 
def simDict(userA):
    sim2 = {}
#     for row in ratings.itertuples(): 
#         if (row.user_id not in sim2) and (row.user_id != userA): #ie iterating over all unique users
#             sim2[row.user_id] = returnSim(userA, row.user_id) 
    for row in users.itertuples():
        if row.user_id != userA:
            sim2[row.user_id] = returnSim(userA, row.user_id)        
    return sim2
def simDict_train(userA,train):
    sim2 = {}
    for row in train.itertuples(): 
        if (row.user_id not in sim2) and (row.user_id != userA): #ie iterating over all unique users
            sim2[row.user_id] = returnSim(userA, row.user_id) 
      
    return sim2



In [14]:
#Returns PREDICTED RATING for a given movie by userA

def returnRating(userA, movie, similarity):
    avgA = ratings.loc[(ratings['user_id']==userA), 'rating'].mean()

    sum_Num = 0.0
    sum_Denom = 0.0
    ctr = 0
    k = 100 #how many top users to consider

    for user_i in similarity:
        if(ctr<k):
            ctr += 1

            r_ip = -1 #rating by this top user for that movie
            try:
                r_ip = ratingDict[(user_i, movie)]
            except:
                pass

            if(r_ip == -1 or math.isnan(r_ip)):
                #ctr-=1
                pass 
            else:
                sum_Num += similarity[user_i] * r_ip
                sum_Denom += abs(similarity[user_i])
        else:
            break
  
    predRating = 0
    try:
        predRating = avgA + (sum_Num/sum_Denom)*(1/k)  
    except:
        pass


    return predRating



In [15]:
#Final user recommender function

def user_recommend(userA, k):

    userRatings = {}  

    similarity = simDict(userA)
    #print(similarity)
    similarity=dict(sorted(similarity.items(), key=lambda item: item[1],reverse=True))
    #similarity=sorted(similarity, key = similarity.get, reverse=True)
    for row in movies.itertuples():
        movieID = row.movie_id
        movieName = movies.loc[(movies['movie_id']==movieID), 'title'].iloc[0]

        old_rating = -1
        try:
            old_rating = ratingDict[(userA, movieID)]
        except:
            pass

        if(old_rating == -1 or math.isnan(old_rating)):
            userRatings[movieName] = returnRating(userA, movieID, similarity)

    moviesSorted = []
    ctr = 0
    for movie in sorted(userRatings, key=userRatings.get, reverse=True):
        if(ctr<k):
            ctr += 1
            moviesSorted.append(movie)
        else:
            break

    for movie in moviesSorted:
        print(movie)


In [16]:
userID_User = input("Enter user ID: ")
k_User = input("Enter number of movies: ")
user_recommend(int(userID_User), int(k_User))

Enter user ID: 18
Enter number of movies: 5
Total Eclipse (1995)
Browning Version, The (1994)
Once Were Warriors (1994)
What Happened Was... (1994)
Underground (1995)


In [19]:
userID_User = input("Enter user ID: ")
k_User = input("Enter number of movies: ")
user_recommend(int(userID_User), int(k_User))

Enter user ID: 6
Enter number of movies: 5
Now and Then (1995)
Mortal Kombat (1995)
Bed of Roses (1996)
Once Upon a Time... When We Were Colored (1995)
Hate (Haine, La) (1995)


# Neural Collaborative Filtering

In [20]:
 
def NCF_Model1(n_features,n_items,n_users):
    
    U=Input(shape=(1,), name='User')
    M=Input(shape=(1,),name='Movie')
    path1=Embedding(n_users,n_features)(U)
    path2=Embedding(n_items,n_features,name='pre-trained')(M)
    ####for intializing with sentence embeddings
    #embedding.set_weights([pretrained_embeddings])
    path1=Reshape((n_features,))(path1)
    path2=Reshape((n_features,))(path2)
    merged=Dot(axes=1)([path1,path2])
    
    #merged= Dense(32,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    #merged= Dense(64,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    #merged= Dense(128,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    #merged= Dense(256,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    #merged= Dense(128,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    #merged= Dense(64,activation='relu', kernel_initializer='he_normal')(merged)
    #merged = BatchNormalization()(merged)
    merged= Dense(32,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    merged= Dense(16,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    merged= Dense(8,activation='relu', kernel_initializer='uniform')(merged)
    #merged = BatchNormalization()(merged)
    merged= Dense(1,activation='sigmoid', kernel_initializer='lecun_uniform')(merged)
    merged=Lambda(lambda x:x*5)(merged)
    
    model = Model(inputs=[U,M], outputs=merged, name='NCF_Model_1')
    return model

def NCF_Model2(n_features,n_items,n_users):
    
    U=Input(shape=(1,), name='User')
    M=Input(shape=(1,),name='Movie')
    
    path1=Embedding(n_users,n_features,name='User_Embedding')(U)
    
    
    path2=Embedding(n_items,n_features,name='Pre-trained_Movie_Embedding')(M)
    
    ####for intializing with sentence embeddings
    #embedding.set_weights([pretrained_embeddings])
    path1=Reshape((n_features,))(path1)
    path1=Dense(256,activation='gelu', kernel_initializer='uniform')(path1)
    path1 = BatchNormalization()(path1)
    path2=Reshape((n_features,))(path2)
    path2=Dense(256,activation='gelu', kernel_initializer='uniform')(path2)
    path2 = BatchNormalization()(path2)
    #merged=(axes=1)([path1,path2])
    #merged = Add()([path1, path2])
    merged = Concatenate()([path1, path2])
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(64,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(32,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(16,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(8,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(1,activation='gelu', kernel_initializer='uniform')(merged)
    
    
    model = Model(inputs=[U,M], outputs=merged, name='NCF_Model_2')
    return model


def NCF_Model3(n_features,n_items,n_users):
    
    U=Input(shape=(1,), name='User')
    M=Input(shape=(1,),name='Movie')
    
    path1=Embedding(n_users,n_features, name='User_Embedding')(U)
    
    
    path2=Embedding(n_items,n_features,name='Pre-trained_Movie_Embedding')(M)
    
    ####for intializing with sentence embeddings
    #path1.set_weights([a])
    path1=Reshape((n_features,))(path1)
    path1=Dense(128,activation='gelu', kernel_initializer='uniform')(path1)
    
    path2=Reshape((n_features,))(path2)
   

    path2=Dense(128,activation='gelu', kernel_initializer='uniform')(path2)
    #merged=(axes=1)([path1,path2])
    merged = Concatenate()([path1, path2])
    #merged=Add()([path1,path2])
    
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged_1 = BatchNormalization()(merged)
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged_1)
    merged_2 = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged_2)
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged_3 = BatchNormalization()(merged)
    merged= Dense(1024,activation='gelu', kernel_initializer='uniform')(merged_3)
    merged = BatchNormalization()(merged)
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 1
    merged= Add()([merged, merged_3])
    
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 2
    merged= Add()([merged, merged_2])
    
    
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 3
    merged=Add()([merged_1,merged])
    
    merged= Dense(64,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(32,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(16,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(8,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(1,activation='gelu', kernel_initializer='uniform')(merged)
    
    
    
    model = Model(inputs=[U,M], outputs=merged, name='NCF_Model_3')
    return model
def NCF_Model6(n_features,n_items,n_users):
    
    U=Input(shape=(1,), name='User')
    M=Input(shape=(1,),name='Movie')
    
    path1=Embedding(n_users,n_features)(U)
    
    
    path2=Embedding(n_items,n_features,name='Pre-trained_Movie_Embedding')(M)
    
    ####for intializing with sentence embeddings
    #path1.set_weights([a])
    path1=Reshape((n_features,))(path1)
    path1=Dense(256,activation='gelu', kernel_initializer='uniform')(path1)
    
    #path2=Reshape((n_features,))(path2)
    path2=Bidirectional(LSTM(256,return_sequences=True))(path2)
    path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=TimeDistributed(Dense(8,activation='gelu', kernel_initializer='uniform'))(path2)
    path2=Bidirectional(LSTM(256))(path2)
    #path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=Dense(256,activation='gelu', kernel_initializer='uniform')(path2)
    #merged=(axes=1)([path1,path2])
    merged = Concatenate()([path1, path2])
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(64,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(32,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(16,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(8,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(1,activation='gelu', kernel_initializer='uniform')(merged)
    #merged=Lambda(lambda x:x*4+1)(merged)
    
    model = Model(inputs=[U,M], outputs=merged, name='NCF_Model_4')
    return model
 
 
def NCF_Model7(n_features,n_items,n_users):
    
    U=Input(shape=(1,), name='User')
    M=Input(shape=(1,),name='Movie')
    
    path1=Embedding(n_users,n_features)(U)
    
    
    path2=Embedding(n_items,n_features,name='Pre-trained_Movie_Embedding')(M)
    
    ####for intializing with sentence embeddings
    #path1.set_weights([a])
    path1=Reshape((n_features,))(path1)
    path1=Dense(256,activation='gelu', kernel_initializer='uniform')(path1)
    
    #path2=Reshape((n_features,))(path2)
    path2=Bidirectional(LSTM(256,return_sequences=True))(path2)
    path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=TimeDistributed(Dense(8,activation='gelu', kernel_initializer='uniform'))(path2)
    path2=Bidirectional(LSTM(256))(path2)
    #path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=Dense(256,activation='gelu', kernel_initializer='uniform')(path2)
    #merged=(axes=1)([path1,path2])
    merged = Concatenate()([path1, path2]) 
    
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged_1 = BatchNormalization()(merged)
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged_1)
    merged_2 = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged_2)
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged_3 = BatchNormalization()(merged)
    merged= Dense(1024,activation='gelu', kernel_initializer='uniform')(merged_3)
    merged = BatchNormalization()(merged)
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 1
    merged= Add()([merged, merged_3])
    
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 2
    merged= Add()([merged, merged_2])
    
    
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 3
    merged=Add()([merged_1,merged])
    
    merged= Dense(64,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(32,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(16,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(8,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(1,activation='gelu', kernel_initializer='uniform')(merged)
    
    
    
    model = Model(inputs=[U,M], outputs=merged, name='NCF_Model_5')
    return model
    
def NCF_Model4(n_features,n_items,n_users):
    
    U=Input(shape=(1,), name='User')
    M=Input(shape=(1,),name='Movie')
    
    path1=Embedding(n_users,n_features)(U)
    
    
    path2=Embedding(n_items,(40*300),name='Pre-trained_Movie_Embedding',)(M)
    
    ####for intializing with sentence embeddings
    #path1.set_weights([a])
    path1=Reshape((n_features,))(path1)
    path1=Dense(256,activation='gelu', kernel_initializer='uniform')(path1)
    
    path2=Reshape((40,300))(path2)
    path2=Bidirectional(LSTM(256,input_shape=(40,300),return_sequences=True))(path2)
    path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=TimeDistributed(Dense(8,activation='gelu', kernel_initializer='uniform'))(path2)
    path2=Bidirectional(LSTM(256))(path2)
    #path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=Dense(256,activation='gelu', kernel_initializer='uniform')(path2)
    #merged=(axes=1)([path1,path2])
    merged = Concatenate()([path1, path2])
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(64,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(32,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(16,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(8,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(1,activation='gelu', kernel_initializer='uniform')(merged)
    
    
    model = Model(inputs=[U,M], outputs=merged, name='NCF_Model_6')
    return model
    
def NCF_Model5(n_features,n_items,n_users):
    
    U=Input(shape=(1,), name='User')
    M=Input(shape=(1,),name='Movie')
    
    path1=Embedding(n_users,n_features)(U)
    
    
    path2=Embedding(n_items,(40*300),name='Pre-trained_Movie_Embedding',)(M)
    
    ####for intializing with sentence embeddings
    #path1.set_weights([a])
    path1=Reshape((n_features,))(path1)
    path1=Dense(256,activation='gelu', kernel_initializer='uniform')(path1)
    
    path2=Reshape((40,300))(path2)
    path2=Bidirectional(LSTM(256,input_shape=(40,300),return_sequences=True))(path2)
    path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=TimeDistributed(Dense(8,activation='gelu', kernel_initializer='uniform'))(path2)
    path2=Bidirectional(LSTM(256))(path2)
    #path2=SeqSelfAttention(attention_type=SeqSelfAttention.ATTENTION_TYPE_MUL,attention_activation='sigmoid')(path2)
    path2=Dense(256,activation='gelu', kernel_initializer='uniform')(path2)
    #merged=(axes=1)([path1,path2])
    merged = Concatenate()([path1, path2]) 
    
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged_1 = BatchNormalization()(merged)
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged_1)
    merged_2 = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged_2)
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged_3 = BatchNormalization()(merged)
    merged= Dense(1024,activation='gelu', kernel_initializer='uniform')(merged_3)
    merged = BatchNormalization()(merged)
    merged= Dense(512,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 1
    merged= Add()([merged, merged_3])
    
    merged= Dense(256,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 2
    merged= Add()([merged, merged_2])
    
    
    merged= Dense(128,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    ###########Skip connection 3
    merged=Add()([merged_1,merged])
    
    merged= Dense(64,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged=Dropout(0.1)(merged)
    merged= Dense(32,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(16,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(8,activation='gelu', kernel_initializer='uniform')(merged)
    merged = BatchNormalization()(merged)
    merged= Dense(1,activation='gelu', kernel_initializer='uniform')(merged)
    
    
    
    model = Model(inputs=[U,M], outputs=merged, name='NCF_Model_7')
    return model

def NCF_Model(model):
    if(model==1):
        return NCF_Model1(1024,3706,6040)
    if(model==2):
        return NCF_Model2(1024,3706,6040)
    if(model==3):
        return NCF_Model3(1024,3706,6040)
    if(model==4):
        return NCF_Model4(300,3706,6040)               
    if(model==5):
        return NCF_Model5(300,3706,6040)
    if(model==6):
        return NCF_Model6(1024,3706,6040)
    if(model==7):
        return NCF_Model7(1024,3706,6040)    



In [21]:
def NCF_recommend(model,user_id,num_movies):
   
    df = pd.DataFrame({"user_id":[user_id for x in range(3706)], "movie_id":[x for x in range(3706)]})
    
    X=df[['user_id','movie_id']].to_numpy()
    X = [X[:, 0], X[:, 1]]
    b=[[user_id for x in range(3706)],[x for x in range(3706)]]
    a=model.predict(X)
    a= pd.DataFrame({'movie_id':df.movie_id.values.flatten(),'user_id':df.user_id.values.flatten(),'rating':a.flatten()})
    a=a.sort_values('rating',ascending=False).iloc[:num_movies]
    preds=[]
    for i in a.movie_id.values:
        preds.append(movies[movies.movie_id==i].title.item())
    return preds

In [24]:
user_id = int(input("Enter user ID: "))
num_movies=int(input("Enter number of movies to be recommended "))
model_num=int(input("Enter your model number "))
m=NCF_Model(model_num)
m.load_weights('Checkpoints/NCF'+str(model_num)+'.h5')

NCF_recommend(m,user_id,num_movies)

Enter user ID: 14
Enter number of movies to be recommended 6
Enter your model number 6
116/116 [==============================] - 2s 6ms/step


['Red Firecracker, Green Firecracker (1994)',
 'Radioland Murders (1994)',
 'I Stand Alone (Seul contre tous) (1998)',
 'Borrowers, The (1997)',
 'Backbeat (1993)',
 'When Night Is Falling (1995)']

# Based on Most-Recently-Watched Movie

In [25]:
def recommend_through_recent(user_id,num_movies):
    user_ratings=ratings.loc[ratings.user_id==user_id]
    user_ratings=user_ratings.sort_values('timestamp',ascending=False).iloc[0]
    movie_emb=movies[movies.movie_id==user_ratings.movie_id.item()].embeddings.item()
    preds=[]
    for row in movies.itertuples():
        preds.append((row.title,cos_sim(movie_emb,row.embeddings)))
        
    preds.sort(key = lambda x: -x[1])
    return [x[0] for x in preds][1:num_movies]

In [26]:
user_id = int(input("Enter user ID: "))
num_movies=int(input("Enter number of movies to be recommended "))
recommend_through_recent(user_id,num_movies)

Enter user ID: 12
Enter number of movies to be recommended 15


['Fall Time (1995)',
 'Thunderbolt and Lightfoot (1974)',
 'I Woke Up Early the Day I Died (1998)',
 'Blue Collar (1978)',
 'Hot Spot, The (1990)',
 'Police Academy 6: City Under Siege (1989)',
 'American Buffalo (1996)',
 'Half Baked (1998)',
 'Bottle Rocket (1996)',
 'Simpatico (1999)',
 'Monty Python and the Holy Grail (1974)',
 'Legal Deceit (1997)',
 'Joyriders, The (1999)',
 'Needful Things (1993)']

# Cold Start 1: Highest rated, most recent movies

In [29]:
#creating list of available genres

tempList = np.ndarray.tolist(movies['genre'].unique())
listGenres = []
maxGenreNum = 0
for genres in tempList:
    s = genres.split(" ")
    maxGenreNum = max(len(s), maxGenreNum)
    for genre in s:
        if(genre not in listGenres): listGenres.append(genre)

listGenres.sort()
print(listGenres)

['Action', 'Adventure', 'Animation', "Children's", 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']


In [30]:
#loading average user ratings and average timestamp of viewing per movie (pickled file)

import pickle
file1 = open("data/avgRatings.pkl",'rb')
avgRatings = pickle.load(file1)

In [32]:
avgRatings.head()
#Avg rating and timestamp

,movie_id,rating,timestamp
0,1.0,4.167235,9.787210e+08
1,2.0,3.292929,9.778557e+08
2,3.0,3.150685,9.776967e+08
3,4.0,2.652174,9.803076e+08
4,5.0,2.951220,9.779067e+08


In [33]:
final_movies_ratings = avgRatings.sort_values(by=['rating', 'timestamp'], ascending=[False, False])
def printTopMovies(final_by_rating, k):
    i = 0
    moviesRet = []
    for row in final_by_rating.itertuples():
        i += 1
        if(i>k):
            break
        else:
            movieID = row.movie_id
            movieName = movies.loc[(movies['movie_id']==movieID), 'title'].iloc[0]
            moviesRet.append(movieName)
  
    for movie in moviesRet:
        print(movie)



In [34]:
num_ret = input("Enter number of movies: ")

printTopMovies(final_movies_ratings, int(num_ret))

Enter number of movies: 14
West Beirut (West Beyrouth) (1998)
Big Combo, The (1955)
Land and Freedom (Tierra y libertad) (1995)
Contempt (Le M�pris) (1963)
Only Angels Have Wings (1939)
Ballad of Narayama, The (Narayama Bushiko) (1958)
Midaq Alley (Callej�n de los milagros, El) (1995)
Autumn Sonata (H�stsonaten ) (1978)
Baby, The (1973)
Johns (1996)
Gate of Heavenly Peace, The (1995)
Onegin (1999)
Jupiter's Wife (1994)
Hearts and Minds (1996)


# Cold Start 2: Based on Favourite Movie

In [36]:
import numpy as np
import pandas as pd
import pickle
import time
#from sentence_transformers import SentenceTransformer
#tokenizer = SentenceTransformer('stsb-roberta-large')
'''
movies=pd.read_csv('COL764-Project/movies_scraped.csv')
movies.description=movies.description+" "+movies.title+" "+movies.genre
#movies['embeddings']=movies.description.apply(lambda x: tokenizer.encode(str(x)))
movies['embeddings']=pickle.load( open( "COL764-Project/embeds.pkl", "rb" ) )
ratings=pd.read_csv('COL764-Project/ratings.csv')
'''
movie_list=movies.title.values
def recommend_through_fav_movie(movie,num_movies):
    
    movie_emb=movies[movies.title==movie].embeddings.item()

    preds=[]
    for row in movies.itertuples():
        
        preds.append((row.title,cos_sim(movie_emb,row.embeddings)))
        
    preds.sort(key = lambda x: -x[1])
    return [x[0] for x in preds][1:num_movies]


    
    

In [37]:
print(movie_list)

['Toy Story (1995)' 'Jumanji (1995)' 'Grumpier Old Men (1995)' ...
 'Tigerland (2000)' 'Two Family House (2000)' 'Contender, The (2000)']


In [38]:
movie=input('Enter your favourite movie from the above list ')
num_movies=int(input('Enter number of movies to be recommended '))
recommend_through_fav_movie(movie,num_movies)

Enter your favourite movie from the above list Jumanji (1995)
Enter number of movies to be recommended 10


['NeverEnding Story III, The (1994)',
 'NeverEnding Story II: The Next Chapter, The (1990)',
 'Jurassic Park (1993)',
 'Pagemaster, The (1994)',
 'Mighty Morphin Power Rangers: The Movie (1995)',
 'Muppets From Space (1999)',
 '3 Ninjas: High Noon On Mega Mountain (1998)',
 'Kazaam (1996)',
 'Virtuosity (1995)']

# Cold Start 3: Based on Favourite Genre

In [39]:
print("List of available genres: ")
for genre in listGenres:
    print(genre)
print("\n")


genreFav = input("Enter your favourite genre: ")
genreList = [genreFav]
movies['genre']=movies['genre'].apply(lambda x: x.split())
moviesRelev = movies[pd.DataFrame(movies.genre.tolist()).isin(genreList).any(1).values]




List of available genres: 
Action
Adventure
Animation
Children's
Comedy
Crime
Documentary
Drama
Fantasy
Film-Noir
Horror
Musical
Mystery
Romance
Sci-Fi
Thriller
War
Western


Enter your favourite genre: Sci-Fi


In [40]:
ratingsGenreSpec = {}

final_movies_ratings = avgRatings.sort_values(by=['rating'], ascending=False)

moviesRelev = moviesRelev.sort_values(by=['movie_id'], ascending = True)
relevMovies = []
prevMovie = -1
for row in moviesRelev.itertuples():
    movieID = row.movie_id
    relevMovies.append(movieID)
    #ratingsGenreSpec[movieID] = avgDict[movieID]
    #then sort this smaller dict by: 
    #ratingsGenreSpec = sorted(ratingsGenreSpec, key=lambda k: (k['starttime'], k['time']))

ratingsRelev = avgRatings[avgRatings['movie_id'].isin(relevMovies)]
finalRatings = ratingsRelev.sort_values(by=['rating', 'timestamp'], ascending=[False, False])

int_inp = input("Enter number of movies: ")
printTopMovies(finalRatings, int(int_inp))

Enter number of movies: 14
Star Wars: Episode IV - A New Hope (1977)
Dr. Strangelove or: How I Learned to Stop Worrying and Love the Bomb (1963)
Matrix, The (1999)
Metropolis (1926)
Fantastic Planet, The (La Plan�te sauvage) (1973)
Star Wars: Episode V - The Empire Strikes Back (1980)
Day the Earth Stood Still, The (1951)
Open Your Eyes (Abre los ojos) (1997)
Delicatessen (1991)
Terminator, The (1984)
City of Lost Children, The (1995)
Blade Runner (1982)
Ghost in the Shell (Kokaku kidotai) (1995)
Solaris (Solyaris) (1972)
